## Genetic Algorithm (GA)

In [36]:
import numpy as np
import random
import glob
import os
import time
import csv
import copy

### Problem and Fitness Functions

In [37]:
def load_problem(file_path):
    """Loads a distance matrix from a .npy file."""
    return np.load(file_path)

def calculate_fitness(individual, dist_matrix):
    """Calculates the total distance of a tour (individual)."""
    fitness = 0
    num_cities = len(individual)
    for i in range(num_cities):
        city1 = individual[i]
        city2 = individual[(i + 1) % num_cities]
        fitness += dist_matrix[city1, city2]
    return fitness

### Initialization (Greedy and Random)

In [38]:
def create_greedy_individual(num_cities, dist_matrix):
    """Creates one individual using the "Nearest Neighbor" heuristic."""
    current_city = random.randrange(num_cities)
    unvisited = set(range(num_cities))
    unvisited.remove(current_city)
    tour = [current_city]
    
    while unvisited:
        nearest_city = min(unvisited, key=lambda city: dist_matrix[current_city, city])
        unvisited.remove(nearest_city)
        tour.append(nearest_city)
        current_city = nearest_city
        
    return tour

def create_random_individual(num_cities):
    """Creates one individual by randomly shuffling the city indices."""
    individual = list(range(num_cities))
    random.shuffle(individual)
    return individual

def initialize_population(pop_size, num_cities, dist_matrix, greedy_ratio=0.8):
    """Initializes the population with 80% greedy seeds and 20% random seeds."""
    population = []
    
    # 80% Greedy Seeds
    num_greedy = int(pop_size * greedy_ratio)
    for _ in range(num_greedy):
        population.append(create_greedy_individual(num_cities, dist_matrix))
        
    # 20% Random Seeds
    num_random = pop_size - num_greedy
    for _ in range(num_random):
        population.append(create_random_individual(num_cities))
        
    return population

### Genetic Operators

In [39]:
# Parent Selection
def tournament_selection(population, fitnesses, k=3):
    """Performs k-tournament selection to choose one parent."""
    participants_indices = random.sample(range(len(population)), k)
    
    best_ind = None
    best_fit = float('inf') # Lower fitness is better
    
    for idx in participants_indices:
        if fitnesses[idx] < best_fit:
            best_fit = fitnesses[idx]
            best_ind = population[idx]
            
    return best_ind

# Crossover
def order_crossover_ox(parent1, parent2):
    """Performs Order Crossover (OX)."""
    size = len(parent1)
    child = [None] * size
    start, end = sorted(random.sample(range(size), 2))
    child[start:end+1] = parent1[start:end+1]
    segment_genes = set(parent1[start:end+1])
    
    p2_genes_to_add = [gene for gene in parent2 if gene not in segment_genes]
            
    p2_idx = 0
    for i in range(size):
        child_idx = (end + 1 + i) % size 
        if child[child_idx] is None:
            child[child_idx] = p2_genes_to_add[p2_idx]
            p2_idx += 1
            
    return child

# Mutation
def swap_mutation(individual):
    """Performs swap mutation by swapping two random positions."""
    ind_copy = individual[:]
    idx1, idx2 = random.sample(range(len(ind_copy)), 2)
    ind_copy[idx1], ind_copy[idx2] = ind_copy[idx2], ind_copy[idx1]
    return ind_copy

### Genetic Algorithm (GA) Solver

In [40]:
def solve_ga(problem_path, pop_size, generations, tournament_size, crossover_rate, mutation_rate, elitism_rate):
    """
    Main function to run the Genetic Algorithm.
    
    Args:
        problem_path (str): Path to the .npy problem file.
        pop_size (int): Population size.
        generations (int): Number of generations to run.
        tournament_size (int): Size for tournament selection.
        crossover_rate (float): Probability of crossover.
        mutation_rate (float): Probability of mutation.
        elitism_rate (float): Percentage of best individuals to carry over.
    """
    
    # Load problem
    dist_matrix = load_problem(problem_path)
    num_cities = dist_matrix.shape[0]
    
    # Initialization
    population = initialize_population(pop_size, num_cities, dist_matrix)
    fitnesses = [calculate_fitness(ind, dist_matrix) for ind in population]
    
    # Keep track of the best solution found so far
    best_overall_fitness = min(fitnesses)
    
    # Generation Loop
    for gen in range(generations):
        
        # Create New Generation
        new_population = []
        
        # Elitism
        # Sort current pop by fitness and add the best to the new generation
        sorted_pop = sorted(zip(population, fitnesses), key=lambda x: x[1])
        num_elites = int(pop_size * elitism_rate)
        
        for i in range(num_elites):
            new_population.append(sorted_pop[i][0])
            
        # Offspring Generation
        # Fill the rest of the population
        num_offspring = pop_size - num_elites
        
        for _ in range(num_offspring):
            # Parent Selection
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            
            # Crossover
            if random.random() < crossover_rate:
                child = order_crossover_ox(parent1, parent2)
            else:
                # Asexual reproduction: clone one parent
                child = parent1[:]
            
            # Mutation
            if random.random() < mutation_rate:
                child = swap_mutation(child)
                
            new_population.append(child)
            
        # Replacement
        # The new population completely replaces the old one
        population = new_population
        fitnesses = [calculate_fitness(ind, dist_matrix) for ind in population]
        
        # Update the best-ever fitness found
        current_best_fitness = min(fitnesses)
        if current_best_fitness < best_overall_fitness:
            best_overall_fitness = current_best_fitness
            
    # Return the best fitness found after all generations
    return best_overall_fitness

### Main Execution Block

In [41]:
# PARAMETERS
POPULATION_SIZE = 100
GENERATIONS = 500
TOURNAMENT_SIZE = 3    
CROSSOVER_RATE = 0.9   
MUTATION_RATE = 0.3    
ELITISM_RATE = 0.1  

# File Paths
PROBLEM_FOLDER = "lab02"
RESULTS_FOLDER = "CSV Results"
# End of Parameters

# Ensure the results directory exists
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Determine output file name
output_file = os.path.join(RESULTS_FOLDER, "ga_results.csv")
print(f"Running Genetic Algorithm (GA) Mode...")
print(f"Params: Pop_Size={POPULATION_SIZE}, Gens={GENERATIONS}, Tourn_Size={TOURNAMENT_SIZE}, "
      f"Cross_Rate={CROSSOVER_RATE}, Mut_Rate={MUTATION_RATE}, Elitism={ELITISM_RATE}\n")

# Find all problem files
problem_files = glob.glob(os.path.join(PROBLEM_FOLDER, "*.npy"))
problem_files.sort() # Sort for consistent processing order

results = []

# Print table header
print(f"{'File Name':<20} | {'Best Fitness':<15} | {'Time (s)':<10}")
print("-" * 49)

# Loop over all problem files and solve them
for file_path in problem_files:
    file_name = os.path.basename(file_path)
    
    start_time = time.time()
    
    best_fitness = solve_ga(
        problem_path=file_path,
        pop_size=POPULATION_SIZE,
        generations=GENERATIONS,
        tournament_size=TOURNAMENT_SIZE,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE,
        elitism_rate=ELITISM_RATE
    )
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Store and print the result for this file
    result_row = {
        "file_name": file_name,
        "best_fitness": best_fitness,
        "time": elapsed_time
    }
    results.append(result_row)
    
    print(f"{file_name:<20} | {best_fitness:<15.2f} | {elapsed_time:<10.4f}")

# Save results to CSV
try:
    with open(output_file, 'w', newline='') as f:
        fieldnames = ["file_name", "best_fitness", "time"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerows(results)
        
    print(f"\n Success! Results saved to '{output_file}'")

except PermissionError:
    print(f"\n ERROR: Permission denied. Could not write to '{output_file}'.")
except Exception as e:
    print(f"\n ERROR: An unexpected error occurred while writing CSV: {e}")

Running Genetic Algorithm (GA) Mode...
Params: Pop_Size=100, Gens=500, Tourn_Size=3, Cross_Rate=0.9, Mut_Rate=0.3, Elitism=0.1

File Name            | Best Fitness    | Time (s)  
-------------------------------------------------
problem_g_10.npy     | 1497.66         | 0.3729    
problem_g_100.npy    | 4341.47         | 1.2154    
problem_g_1000.npy   | 14060.33        | 13.8574   
problem_g_20.npy     | 1755.51         | 0.4805    
problem_g_200.npy    | 6318.06         | 2.2618    
problem_g_50.npy     | 2880.75         | 0.7555    
problem_g_500.npy    | 9692.40         | 6.1487    
problem_r1_10.npy    | 184.27          | 0.3896    
problem_r1_100.npy   | 746.97          | 1.2637    
problem_r1_1000.npy  | 2552.25         | 14.1457   
problem_r1_20.npy    | 340.86          | 0.4684    
problem_r1_200.npy   | 1112.71         | 2.2447    
problem_r1_50.npy    | 579.56          | 0.7475    
problem_r1_500.npy   | 1753.46         | 6.3148    
problem_r2_10.npy    | -411.70         | 0